<a href="https://colab.research.google.com/github/nirb28/nn_catalyst/blob/main/src/pl/scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
%%capture
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    print("Running in Colab!")
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    !pip install pytorch_lightning
    !pip install torchmetrics
else:
    print("Not running in Colab.")

In [12]:
from pytorch_lightning.loggers import TensorBoardLogger
import torch
DEBUG = False
# Training hyperparameters
INPUT_SIZE = 1479
NUM_TARGETS = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 512
NUM_EPOCHS = 1 #150
NUM_WORKERS = 0
# Compute related
ACCELERATOR = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICES = [0]
PRECISION = 32
CHECKPOINTS_FOLDER = "/checkpoints/stn_2_r1"

In [19]:
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import random_split
import pytorch_lightning as pl
import torch, math, os
from torch.utils.data import Dataset, DataLoader
import numpy as np

seed = 1234
pl.seed_everything(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

def resolve_path_gdrive(relativePath):
    if os.path.exists('/content/drive'):
        return '/content/drive/MyDrive/work/gdrive-workspaces/git/nn_catalyst/' + relativePath
    else:
        from utils import get_project_root
        return get_project_root() + "/" + relativePath

print(f"Root project folder is at {resolve_path_gdrive('.')}")

INFO:lightning_fabric.utilities.seed:Seed set to 1234


Root project folder is at /content/drive/MyDrive/work/gdrive-workspaces/git/nn_catalyst/.


In [3]:
#%load_ext tensorboard
#%tensorboard --logdir f"/content/drive/MyDrive/work/gdrive-workspaces/git/nn_catalyst/checkpoints/{CHECKPOINTS_FOLDER}/lightning_logs"

In [6]:
import numpy as np
datafile='src/pl/merged_data_last29_reordered_byR2.csv'
xy_orig = np.loadtxt(resolve_path_gdrive(datafile), delimiter=',', skiprows=1, dtype=float, max_rows=None)

In [23]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
class CatalystDataset(Dataset):
    def __init__(self, target_num, datafile='src/pl/merged_data_last29_reordered_byR2.csv',
                 useTargetsAsFeatures=False):
        # Initialize data, download, etc.
        #xy = np.loadtxt(resolve_path_gdrive(datafile), delimiter=',', skiprows=1, dtype=float, max_rows=None)
        xy = xy_orig.copy()
        #xy = StandardScaler().fit_transform(xy)
        self.n_samples = xy.shape[0]
        # here the first column is the class label, the rest are the features
        total_targets = 29
        target_col_start = -(total_targets - target_num + 1)
        if useTargetsAsFeatures:
            self.x_data = torch.from_numpy(xy[:,:target_col_start]).float()  # size [n_samples, n_features]
        else:
            self.x_data = torch.from_numpy(xy[:,:-total_targets]).float()  # size [n_samples, n_features]
        # get only first column as y
        self.y_data = torch.unsqueeze(torch.from_numpy(xy[:,target_col_start:][:,0]), 1).float()  # size [n_samples, 1]

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]
        return sample

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

class CatalystDataModule(pl.LightningDataModule):
    def __init__(self, data_dir, batch_size, num_workers, target_num):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.entire_dataset = CatalystDataset(target_num=target_num,
                                              useTargetsAsFeatures=True)
        self.target_num = target_num

    def prepare_data(self):
        pass

    def setup(self, stage=None):
        train_set_size = int(len(self.entire_dataset) * .8)
        test_set_size = int(len(self.entire_dataset) * 0.1)
        valid_set_size = len(self.entire_dataset) - train_set_size - test_set_size
        print(f"Train set size: {train_set_size}, Test set size: {test_set_size}, Valid set size: {valid_set_size}")
        xy = StandardScaler().fit_transform(xy)
        self.train_ds, self.val_ds, self.test_ds = random_split(
            self.entire_dataset, [train_set_size, valid_set_size, test_set_size]) #, generator=torch.Generator().manual_seed(42))
        return

    def train_dataloader(self):
        return DataLoader(
            self.train_ds.dataset, batch_size=self.batch_size,
            num_workers=self.num_workers, shuffle=False,
        )
    def val_dataloader(self):
        return DataLoader(
            self.val_ds.dataset, batch_size=self.batch_size,
            num_workers=self.num_workers, shuffle=False,
        )
    def test_dataloader(self):
        return DataLoader(
            self.test_ds.dataset, batch_size=self.batch_size,
            num_workers=self.num_workers, shuffle=False,
        )
dm1 = CatalystDataModule(
        data_dir="",
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        target_num=29
    )
dm1.setup()
dm1.prepare_data()

Train set size: 20986, Test set size: 2623, Valid set size: 2624


In [24]:
class BaseModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.r2 = torchmetrics.R2Score()
        self.loss_fn = nn.MSELoss()
        self.validation_step_outputs = []

    def training_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        self.log_dict(
            {
                "train_loss": loss,
            },
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        accuracy = self.r2(scores, y)
        self.log("train_acc", accuracy, prog_bar=True)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        self.log("val_loss", loss)
        self.validation_step_outputs.append(loss)
        return loss

    def on_validation_epoch_end(self):
        epoch_average = torch.stack(self.validation_step_outputs).mean()
        self.log("validation_epoch_average", epoch_average)
        self.validation_step_outputs.clear()  # free memory

    def test_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        self.log("test_loss", loss)
        return loss

    def _common_step(self, batch, batch_idx):
        x, y = batch
        x = x.reshape(x.size(0), -1)
        scores = self.forward(x)
        loss = self.loss_fn(scores, y)
        if DEBUG == True:
            print(f"loss: {loss}, len: {len(y)}")
        return loss, scores, y

    def predict_step(self, batch, batch_idx):
        x, y = batch
        x = x.reshape(x.size(0), -1)
        scores = self.forward(x)
        preds = torch.argmax(scores, dim=1)
        return preds

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(lr=self.lr, params=self.parameters())
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, min_lr=0.000000001, threshold=0.001)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}

class SingleTargetNet(BaseModel):

    def __init__(self, input_size=INPUT_SIZE, learning_rate=0.001, dropout_rate=0.5, target=1):
        super(SingleTargetNet, self).__init__()
        self.lr = learning_rate
        self.loss_fn = nn.MSELoss()

        self.fc1 = nn.Linear(input_size, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 1)
        self.fc_skip = nn.Linear(1024, 512)
        self.dropout = nn.Dropout(dropout_rate)
        self.save_hyperparameters()

    def forward(self, x):
        x1 = F.relu(self.bn1(self.fc1(x)))
        x1 = self.dropout(x1)

        x2 = F.relu(self.bn2(self.fc2(x1)))
        x2 = self.dropout(x2)

        # Skip connection
        x2 += self.fc_skip(x1)

        x3 = self.fc3(x2)
        return x3


In [26]:
from torch import nn, optim
import torchmetrics
from torchmetrics.regression import R2Score
import torch.nn.functional as F
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.callbacks import RichProgressBar

torch.set_float32_matmul_precision("medium") # to make lightning happy

if __name__ == "__main__":
    # do everything in a loop for all the targets
    for target in range(1, 2):
        print(f"Target: {target}")
        tensorboard = TensorBoardLogger(resolve_path_gdrive(f'{CHECKPOINTS_FOLDER}/lightning_logs'), name=f"{target}")
        dm = CatalystDataModule(
            data_dir="",
            batch_size=BATCH_SIZE,
            num_workers=NUM_WORKERS,
            target_num=target
        )
        dm.setup()
        dm.prepare_data()
        model = SingleTargetNet (
            input_size=dm.entire_dataset.x_data.shape[1],
            learning_rate=LEARNING_RATE,
        )
        checkpoint_callback = ModelCheckpoint(
            dirpath=resolve_path_gdrive(f'{CHECKPOINTS_FOLDER}/{target}'),
            filename='{epoch:02d}-{val_loss:.2f}',
            save_top_k=1,
            verbose=True,
            monitor='val_loss',
            mode='min'
        )
        lr_monitor = LearningRateMonitor(logging_interval='step')
        trainer = pl.Trainer(
            accelerator=ACCELERATOR,
            devices=1,
            min_epochs=1,
            max_epochs=NUM_EPOCHS,
            precision=PRECISION,
            fast_dev_run=False,
            enable_checkpointing=True,
            enable_progress_bar=True,
            log_every_n_steps=20,
            logger=tensorboard,
            callbacks=[checkpoint_callback, lr_monitor, RichProgressBar(),
                    EarlyStopping(monitor="train_loss", patience=10, verbose=True, mode="min")]
        )
        trainer.fit(model, dm)
        trainer.validate(model, dm)
        trainer.test(model, dm)

Target: 1


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Train set size: 20986, Test set size: 2623, Valid set size: 2624
Train set size: 20986, Test set size: 2623, Valid set size: 2624


┏━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name    ┃ Type        ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ r2      │ R2Score     │      0 │ train │
│ 1 │ loss_fn │ MSELoss     │      0 │ train │
│ 2 │ fc1     │ Linear      │  1.5 M │ train │
│ 3 │ bn1     │ BatchNorm1d │  2.0 K │ train │
│ 4 │ fc2     │ Linear      │  524 K │ train │
│ 5 │ bn2     │ BatchNorm1d │  1.0 K │ train │
│ 6 │ fc3     │ Linear      │    513 │ train │
│ 7 │ fc_skip │ Linear      │  524 K │ train │
│ 8 │ dropout │ Dropout     │      0 │ train │
└───┴─────────┴─────────────┴────────┴───────┘

Trainable params: 2.6 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 2.6 M                                                                                                
Total estimated model params size (MB): 10                                                                         
Modules in train mode: 9                                                                                           
Modules in eval mode: 0

...

Train set size: 20986, Test set size: 2623, Valid set size: 2624


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │         5374885.5         │
│ validation_epoch_average  │         5325029.5         │
└───────────────────────────┴───────────────────────────┘

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │         5374885.5         │
└───────────────────────────┴───────────────────────────┘

In [30]:
from torchmetrics import R2Score
from torchmetrics import MeanSquaredError

def r2scoreAndMSE(model, dataloader):
    r2_score_metric = R2Score()
    mse = MeanSquaredError()
    for batch_idx, (data, target) in enumerate(dataloader):
        predictions = model(data)
        #import pdb; pdb.set_trace()
        r2_score_metric.update(predictions, target)
        mse.update(predictions, target)
    return r2_score_metric.compute().detach().item(), mse.compute().detach().item()

In [ ]:
from pathlib import Path
import pandas as pd
from itertools import chain

results = []
if __name__ == "__main__":
    # do everything in a loop for all the targets
    for target in range(1, 30):
        print(f"Target: {target}")
        dm = CatalystDataModule(
            data_dir="",
            batch_size=BATCH_SIZE,
            num_workers=NUM_WORKERS,
            target_num=target
        )
        dm.prepare_data()
        dm.setup()
        if DEBUG == True:
            print(f"Train set size: {len(dm.train_ds),dm.train_ds.dataset.x_data.shape[1]}")
            print(f"Test set size: {len(dm.test_ds)}, Valid set size: {len(dm.val_ds)}")

        checkpoint_path=resolve_path_gdrive(f'{CHECKPOINTS_FOLDER}/{dm.target_num}')
        pathlist = Path(checkpoint_path).glob('**/*.ckpt')
        for path in pathlist:
            # because path is object not string
            model = SingleTargetNet.load_from_checkpoint(str(path))
            model.eval()
            model.cpu()
            # add row to resultsDF
            train_r2, train_mse = r2scoreAndMSE(model, dm.train_dataloader())
            val_r2, val_mse = r2scoreAndMSE(model, dm.val_dataloader())
            test_r2, test_mse = r2scoreAndMSE(model, dm.test_dataloader())

            results.append([target, os.path.basename(path), train_r2, train_mse, val_r2, val_mse, test_r2, test_mse])


Target: 1
Train set size: 20986, Test set size: 2623, Valid set size: 2624
Target: 2
Train set size: 20986, Test set size: 2623, Valid set size: 2624
Target: 3
Train set size: 20986, Test set size: 2623, Valid set size: 2624
Target: 4
Train set size: 20986, Test set size: 2623, Valid set size: 2624
Target: 5
Train set size: 20986, Test set size: 2623, Valid set size: 2624
Target: 6
Train set size: 20986, Test set size: 2623, Valid set size: 2624
Target: 7
Train set size: 20986, Test set size: 2623, Valid set size: 2624
Target: 8
Train set size: 20986, Test set size: 2623, Valid set size: 2624
Target: 9
Train set size: 20986, Test set size: 2623, Valid set size: 2624
Target: 10
Train set size: 20986, Test set size: 2623, Valid set size: 2624
Target: 11
Train set size: 20986, Test set size: 2623, Valid set size: 2624
Target: 12
Train set size: 20986, Test set size: 2623, Valid set size: 2624
Target: 13
Train set size: 20986, Test set size: 2623, Valid set size: 2624
Target: 14
Train set 

In [ ]:
resultsDFcolumns = ["Target", "ModelFile", "Train R2", "Train MSE", "Val R2", "Val MSE", "Test R2", "Test MSE"]
resultsDF = pd.DataFrame(results, columns=resultsDFcolumns)
resultsDF

,Target,ModelFile,Train R2,Train MSE,Val R2,Val MSE,Test R2,Test MSE
0,1,epoch=56-val_loss=0.00.ckpt,0.998835,0.001165,0.998835,0.001165,0.998835,0.001165
1,2,epoch=97-val_loss=0.00.ckpt,0.998941,0.001059,0.998941,0.001059,0.998941,0.001059
2,3,epoch=69-val_loss=0.00.ckpt,0.998975,0.001025,0.998975,0.001025,0.998975,0.001025
3,4,epoch=86-val_loss=0.00.ckpt,0.999022,0.000978,0.999022,0.000978,0.999022,0.000978
4,5,epoch=91-val_loss=0.00.ckpt,0.999025,0.000975,0.999025,0.000975,0.999025,0.000975
5,6,epoch=80-val_loss=0.00.ckpt,0.999070,0.000930,0.999070,0.000930,0.999070,0.000930
6,7,epoch=148-val_loss=0.02.ckpt,0.983475,0.016525,0.983475,0.016525,0.983475,0.016525
7,8,epoch=146-val_loss=0.02.ckpt,0.983833,0.016167,0.983833,0.016167,0.983833,0.016167
8,9,epoch=144-val_loss=0.01.ckpt,0.987937,0.012063,0.987937,0.012063,0.987937,0.012063
9,10,epoch=58-val_loss=0.03.ckpt,0.965896,0.034103,0.965896,0.034103,0.965896,0.034103


In [ ]:
resultsDF.to_csv(resolve_path_gdrive(f'{CHECKPOINTS_FOLDER}/results.csv'), index=False)

In [ ]:
dm1 = CatalystDataModule(
        data_dir="",
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        target_num=target
    )
dm1.prepare_data()
dm1.setup()

print(r2scoreAndMSE(model, dm1.train_dataloader()))
print(r2scoreAndMSE(model, dm1.val_dataloader()))
print(r2scoreAndMSE(model, dm1.test_dataloader()))

In [ ]:
import numpy as np
matrix = np.array([[1, 2, 3, 4],
                   [5, 6, 7, 8],
                   [9, 10, 11, 12]])
m = np.delete(matrix, [-1], axis=1)
m